### Imports

In [1]:
import time
import json
import random
import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

### Utils

In [2]:
def get_random_string(k: int = 20):
    return "".join(random.choices("abcdefghijklmnopqrstuvwxyz", k=k))

def get_get_random_json():
    return {
        "str1": get_random_string(),
        "str2": get_random_string(),
        "int1": random.randint(0, 10_000),
        "int2": random.randint(-10_000, 0),
        "alphanumeric": f"{random.randint(0, 100_000)}" + get_random_string(),
    }

### Requests

In [3]:
def hello_world_request(_):
    return requests.get("http://localhost:80/hello_world").content

def route_test_request(args):
    return requests.get(f"http://localhost:80/{args[0]}/{args[1]}").content

def json_parsing_request(payload):
    return requests.post("http://localhost:80/json", json=payload).content

In [4]:
num_threads = [1, 2, 4, 8]
num_reqs = [1_000, 2_000, 4_000, 8_000]

In [5]:
# results => tuple(task, threads, reqs, time)
results = []
for nt in num_threads:
    for nr in num_reqs:
        # hello world
        with ThreadPoolExecutor(nt) as ex:
            start = time.perf_counter()
            res = ex.map(hello_world_request, range(nr))
        stop = time.perf_counter()
        print(f"Served {nr:,} in: {(stop-start)*1_000}ms")
        results.append(("hello_world", nt, nr, (stop-start)*1_000))

        # route testing
        req_stuff = [(random.randint(0, 10_000), get_random_string()) for _ in range(nr)]
        with ThreadPoolExecutor(nt) as ex:
            start = time.perf_counter()
            res = ex.map(route_test_request, req_stuff)
        stop = time.perf_counter()
        print(f"Served {nr:,} in: {(stop-start)*1_000}ms")
        results.append(("route_testing", nt, nr, (stop-start)*1_000))

        # json parsing
        req_stuff = [(random.randint(0, 10_000), get_random_string()) for _ in range(nr)]
        with ThreadPoolExecutor(nt) as ex:
            start = time.perf_counter()
            res = ex.map(route_test_request, req_stuff)
        stop = time.perf_counter()
        print(f"Served {nr:,} in: {(stop-start)*1_000}ms")
        results.append(("json_parsing", nt, nr, (stop-start)*1_000))

Served 1,000 in: 2401.3422499999997ms
Served 1,000 in: 2445.2992499999996ms
Served 1,000 in: 2345.7852499999995ms
Served 2,000 in: 4714.365125ms
Served 2,000 in: 4886.103041999999ms
Served 2,000 in: 4536.100208000001ms
Served 4,000 in: 9113.10625ms
Served 4,000 in: 10482.943833ms
Served 4,000 in: 10095.155334000005ms
Served 8,000 in: 20681.365707999994ms
Served 8,000 in: 20580.276958ms
Served 8,000 in: 20488.983208999998ms
Served 1,000 in: 1837.7534170000017ms
Served 1,000 in: 1608.0085410000038ms
Served 1,000 in: 1910.2836250000053ms
Served 2,000 in: 3495.874959000005ms
Served 2,000 in: 3256.583375000005ms
Served 2,000 in: 3573.2413329999986ms
Served 4,000 in: 7274.909124999994ms
Served 4,000 in: 8080.458165999999ms
Served 4,000 in: 8699.715541000018ms
Served 8,000 in: 15494.093666999986ms
Served 8,000 in: 15278.929832999978ms
Served 8,000 in: 15908.391915999999ms
Served 1,000 in: 1487.0450000000233ms
Served 1,000 in: 1534.5036669999956ms
Served 1,000 in: 1701.4694999999733ms
Served 2

In [6]:
# write out 
df = pd.DataFrame(results)
df.columns = ["task", "threads", "requests", "time(ms)"]
df.to_csv("../results/axum.csv")